In [1]:
import requests

api_key = '65c931bf3d4f99dfdcfe83fb7a16fac8'
city = 'Dublin,IE'

url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units="metric"'

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print('Failed to retrieve weather data')

{'coord': {'lon': -6.2672, 'lat': 53.344}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 277.36, 'feels_like': 275.53, 'temp_min': 276.25, 'temp_max': 278.15, 'pressure': 1029, 'humidity': 73}, 'visibility': 10000, 'wind': {'speed': 2.06, 'deg': 250}, 'clouds': {'all': 75}, 'dt': 1677968115, 'sys': {'type': 2, 'id': 2037117, 'country': 'IE', 'sunrise': 1677913613, 'sunset': 1677953232}, 'timezone': 0, 'id': 2964574, 'name': 'Dublin', 'cod': 200}


In [2]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import simplejson as json
import requests
import time
from IPython.display import display

In [3]:
URI="dbbikes.chdxsa0c6itr.eu-west-1.rds.amazonaws.com"
PORT=3306
DB="dbbikes"
USER="admin"
PASSWORD="xiaoxin123"
engine=create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER,PASSWORD,URI,PORT,DB),echo=True)

In [4]:
sql="""
Create database if not exists dbbikes;
"""
engine.execute(sql)

2023-03-04 22:18:36,545 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-03-04 22:18:36,547 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-04 22:18:36,975 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-03-04 22:18:36,978 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-04 22:18:36,985 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-03-04 22:18:36,987 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-04 22:18:37,007 INFO sqlalchemy.engine.Engine 
Create database if not exists dbbikes;

2023-03-04 22:18:37,008 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-04 22:18:37,017 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
# create table
sql="""
create table if not exists weather(
number int primary key,
weather varchar(64),
temp float,
feel_temp float,
min_temp float,
max_temp float,
pressure int,
humidity int,
visibility int,
wind_speed float,
wind_deg int,
time varchar(64))
"""
try:
    res=engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-03-04 22:18:37,047 INFO sqlalchemy.engine.Engine 
create table if not exists weather(
number int primary key,
weather varchar(64),
temp float,
feel_temp float,
min_temp float,
max_temp float,
pressure int,
humidity int,
visibility int,
wind_speed float,
wind_deg int,
time varchar(64))

2023-03-04 22:18:37,049 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-04 22:18:37,058 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [6]:
import requests
import traceback
import datetime
import time
import json
api_key = '65c931bf3d4f99dfdcfe83fb7a16fac8'
city = 'Dublin,IE'
url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units="metric"'

In [7]:
def writeWeather_to_file(r):
    now=datetime.datetime.now()
    with open("weather/weather_{}".format(now).replace(" ","_"),"w") as f:
        f.write(str(r))

In [8]:
def weather_to_db(text,number):
    weather=text
    now=datetime.datetime.now()
    print(type(weather),len(weather))
    print(weather)
    print(weather.get("weather")[0].get("main"))
    weather_vals=(number,weather.get("weather")[0].get("main"),weather.get("main").get("temp"),weather.get("main").get("feels_like"),weather.get("main").get("temp_min"),weather.get("main").get("temp_max"),weather.get("main").get("pressure"),weather.get("main").get("humidity"),weather.get("visibility"),weather.get("wind").get("speed"),weather.get("wind").get("deg"),now)
    engine.execute("insert into weather values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",weather_vals)
    return


In [10]:
def getData():
    try:
        now=datetime.datetime.now()
        r= requests.get(url)
        r=r.json()
        print(r)
        writeWeather_to_file(r)
        weather_to_db(r,number)
    except:
        print(traceback.format_exc())
    return
number=1
while True:
    number+=1
    getData()
    time.sleep(2*60*60)
    

{'coord': {'lon': -6.2672, 'lat': 53.344}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 277.36, 'feels_like': 275.53, 'temp_min': 276.25, 'temp_max': 278.15, 'pressure': 1029, 'humidity': 73}, 'visibility': 10000, 'wind': {'speed': 2.06, 'deg': 250}, 'clouds': {'all': 75}, 'dt': 1677968115, 'sys': {'type': 2, 'id': 2037117, 'country': 'IE', 'sunrise': 1677913613, 'sunset': 1677953232}, 'timezone': 0, 'id': 2964574, 'name': 'Dublin', 'cod': 200}
<class 'dict'> 13
{'coord': {'lon': -6.2672, 'lat': 53.344}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 277.36, 'feels_like': 275.53, 'temp_min': 276.25, 'temp_max': 278.15, 'pressure': 1029, 'humidity': 73}, 'visibility': 10000, 'wind': {'speed': 2.06, 'deg': 250}, 'clouds': {'all': 75}, 'dt': 1677968115, 'sys': {'type': 2, 'id': 2037117, 'country': 'IE', 'sunrise': 1677913613, '

KeyboardInterrupt: 